# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** It is a classification problem because the output of the algorithmn, given students as data points, should be binary: whether the student may successfully graduate or not and consequently whether he or she needs or do not need early intervention.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = (student_data[student_data['passed'] == 'yes']).shape[0]

# TODO: Calculate failing students
n_failed = (student_data[student_data['passed'] == 'no']).shape[0]

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/n_students*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()


Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))



Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
from sklearn import cross_validation

num_train = 300
num_test = X_all.shape[0] - num_train
    
# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X_all, y_all, train_size=num_train, stratify=y_all, random_state = 222)

print "% Pass [train]: {}".format(len(y_train[y_train == 'yes']) / float(y_train.shape[0]) * 100)
print "% Pass [test]: {}".format(len(y_test[y_test == 'yes']) / float(y_test.shape[0]) * 100)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])



% Pass [train]: 67.0
% Pass [test]: 67.3684210526
Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** 

- Support Vector Machine
- Decision Tree Ensemble: Random Forest
- Logistic Regression

#### SVM

- General application: 
  - A class of learning methods used for both classification and regression. 
  - Applications include text-mining, facial recognition and image-processing.
- Advantages:
  - Naturally resistent to over-fitting, as long as the kernel and kernel parameters are tuned well, even in the scenario where the dimensionality of the data is relatively high compared to the number of samples.
  - Memory efficient
- Disadvantages: 
  - SVMs are generally slow to train. With the default, general-purpose non-linear SVC implementation libSVM, the time complexity for training is **~O(n^2)**. 
  - Difficult to do *incremental learning* with a SVM
  - Black-boxed and hard to interpret results unlike *plain* decision trees
  - Unable to deal with multi-class classifications

#### Decision Tree Ensemble: Random Forest

- General application: 
  - An ensemble learning method comprising of multiple decision trees. 
  - Applications include credit scoring, company strategy selection (risk analysis etc)
- Advantages:
  - Avoids overfitting issues present with decision trees by averaging deep trees
  - Reduces variance by averaging out individual values that may be stuck in local minima
  - In the process, also automatically determines which subset of features are important.
- Disadvantages: 
  - Memory usage
  - Loss of interpretability over *plain* decision trees
  - Most implementations are slow when you have a big number of features: **~O(v\*nlogn)** 
  - Bad at generalizing when there is completely *new* data in the features

#### Logistic Regression

- General application: 
  - A special case of generalized linear model that is primarily only used for classification (despite the name)
  - Widely used in medical sciences to predict mortality for injured patients, risk of strokes etc. 
  - In general also used to find the probablity of X given multiple features/variables as in multivariate logistic regressions.
- Advantages:
  - A well-behaved classification model that is good to start with to use as a baseline
  - Simple yet don't need to worry about feature correlation (vs Naive Bayes)
  - Able to output a probabilistic interpretation
  - Able to do incremental learning 
- Disadvantages: 
  - Sensitive to choice of features; bad results if the wrong features are chosen.
  - Tend to overfit, overstating the accuracy of its predictions

#### Choice

The number of training data sample is relativey few whereas the dimensionality of the data is rather high. That is the reason I chose SVM as the first model as SVMs generally performs well even in high-dimensionality. Computationally, it is on the high side at **~O(n^2)** but it remains to be seen how much higher it is compared to the other models.

The second model I chose is an ensemble of decision trees: Random Forest. Decision trees in general do not have the constraint of needing linearly separable datapoints and are (generally) fast but tends to overfit. Random forest ensemble method helps with the overfitting issue and generally does well in alot of data sets so I felt that it should be evaluated to see how well it would perform in this case.

Lastly, I wanted to also include a simpler model: the logistic regression because of the limitations in resources. It somewhat belongs to a different *type* of algorithm compared to the other two in the sense that it is just a statistical regression model. It is simple to visualize, efficient and provides a pretty good baseline to compare other algorithms with.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [10]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = SVC(random_state=111)
clf_B = RandomForestClassifier(random_state=222)
clf_C = LogisticRegression(random_state=333)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
def train_predict_sizes(clf, train_sizes):
    for train_size in train_sizes: 
        train_predict(clf, X_train[:train_size], y_train[:train_size], X_test, y_test)

print "---- SVC ----"
train_predict_sizes(clf_A, [100, 200, 300])

print "\n---- Random Forest ----"
train_predict_sizes(clf_B, [100, 200, 300])

print "\n---- Logistic Regresion ----"
train_predict_sizes(clf_C, [100, 200, 300])


---- SVC ----
Training a SVC using a training set size of 100. . .
Trained model in 0.0100 seconds
Made predictions in 0.0029 seconds.
F1 score for training set: 0.8392.
Made predictions in 0.0026 seconds.
F1 score for test set: 0.7862.
Training a SVC using a training set size of 200. . .
Trained model in 0.0090 seconds
Made predictions in 0.0074 seconds.
F1 score for training set: 0.8630.
Made predictions in 0.0034 seconds.
F1 score for test set: 0.7973.
Training a SVC using a training set size of 300. . .
Trained model in 0.0143 seconds
Made predictions in 0.0081 seconds.
F1 score for training set: 0.8621.
Made predictions in 0.0026 seconds.
F1 score for test set: 0.8054.

---- Random Forest ----
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.2023 seconds
Made predictions in 0.0022 seconds.
F1 score for training set: 0.9573.
Made predictions in 0.0017 seconds.
F1 score for test set: 0.6609.
Training a RandomForestClassifier using a training

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVC**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0029           |        0.0026          |      0.8392      |    0.7862       |
| 200               |        0.0074           |        0.0034          |      0.8630      |    0.7973       |
| 300               |        0.0081           |        0.0026          |      0.8621      |    0.8054       |

** Classifer 2 - Random Forest**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0022            |         0.0017         |      0.9573      |     0.6609      |
| 200               |       0.0019            |         0.0018         |      0.9961      |     0.7967      |
| 300               |       0.0022            |         0.0022         |      0.9975      |     0.7231      |

** Classifer 3 - Logistic Regression**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0003            |         0.0002         |      0.8661      |     0.7846      |
| 200               |       0.0003            |         0.0002         |      0.8291      |     0.7463      |
| 300               |       0.0005            |         0.0003         |      0.8406      |     0.7519      |


## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **  If we're only evaluating based on absolute accuracy, I believe the SVM model will end up getting a better score than the other two approaches after fine tuning. However, given the constraints of limited resources, I think we should favor simpler approaches. The SVM approach take significant computation to train and predict as compared to the other 2 models. 

Judging from the how the random forest is over-fitting on the training data while still having a relatively low test score, I feel that that I'll probably need to tune the model by increasing the number of trees involved via *n_estimators* and consequently the complexity of the random forest which means the time-complexity will surely increase: **O(v\*nlog(n))** The viability of the model, given our limited computation resources, is lowered because of this. 

Logistic regression is the simplest model to visualize for the board of directors (the other two are essentially blackboxes. It is also extremely efficient and consumes very little computational power and memory to train and predict compared to the other two more complex algorithms. The results on the test set seem reasonably close to the other two algorithms after fine-tuning too. Also, as the training set is expected to grow steadily over a decade, overfitting becomes less of a issue for logistic regression eventually, thus the results will only get more accurate. On top of that, the school can also back-track and find the estimated regression coefficients of a log scale and interpret the effects of each feature in future to help with developing the intervention.

**Therefore, I would argue that the *most appropriate* model and a reasonable compromise between accuracy and resource-usage in this case would be the Logistic Regression.**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: ** Logistic regression works similar to a linear regression model but is used only when the dependent variable is categorical, e.g. whether a student passes or fails. There is a *link* function which is defined as the log of odds: ln(P/1-P) where P is the probability of a student passing. This provides the model with the continuous value that is used for standard linear regression. Fitting the data refers to the same as linear regression analysis e.g. minimizing the error between expected value and actual via the *link* function. This yields the coefficients, or significance of each feature and a predictor function (fitted model). When making a prediction, the value obtained via the predictor function is converted back via an exponential function (reverse of log) to obtain the probability that the expected value is a 1 or 0 (pass or fail). 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [45]:
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.cross_validation import KFold

parameters = {'C': [0.001, 0.005, 0.0075, 0.01, 0.05, 0.1, 1, 10, 100] }

clf = LogisticRegression()

def performance_metric(label, prediction):
    return f1_score(label, prediction, pos_label='yes')

f1_scorer = make_scorer(performance_metric)

grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)
grid_obj.fit(X_train, y_train)
print "Best Params: " + str(grid_obj.best_params_)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))


Best Params: {'C': 0.05}
Made predictions in 0.0007 seconds.
Tuned model has a training F1 score of 0.8348.
Made predictions in 0.0003 seconds.
Tuned model has a testing F1 score of 0.8138.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final model's F1 score is 0.8348 for training and 0.8138 for testing. We can see a slight drop in the score for the training set, but significantly better test score compared to the untuned model. 

This is expected behaviour as the final model has a higher regularization strength (lower C value); this reduces over-fitting, which is a common problem for a low bias/high variance model such as logistic regression which in turn improves the generalization capability of the model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.